# Création du jeu de données

- Villes : Amsterdam, Copenhagen, Madrid, Paris, Rome, Sofia, Valletta, Vienna et Vilnius.

- Date : le nombre de jour entre le moment où vous regardez sur le site et le jour du check-in. Entre 44 et 0.

- Language : austrian, belgian, bulgarian, croatian, cypriot, czech, danish, dutch, estonian, finnish, french, german, greek, hungarian, irish, italian, latvian, lithuanian, luxembourgish, maltese, polish, portuguese, romanian, slovakian, slovene, spanish, swedish.

- mobile : 1 (téléphone) ou 0 (ordinateur).
 
- avatar : Identifiant d’un utilisateur fictif 



In [1]:
import urllib.parse
import requests
import pandas as pd
import random

In [2]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [3]:
user_id = 'da2c83c3-98b2-43da-8389-3f54e9d49b64'

<div class="alert alert-success">
<b>Premier test</b>
</div>

On cherche à tester l'influence de la langue de l'interface : le prix est-il influencé si on prend une langue différente de la langue du pays où se trouve l'hotel ?
On garde un même utilisateur, on choisit aléatoirement téléphone ou ordinateur. Pour chaque ville on teste la langue locale (ou majoritairement parlée) ainsi qu'une autre langue de façon aléatoire et on teste pour chaque jour entre j-14 et j-1.

In [4]:
#Création de l'avatar
name = 'avatar1'
r = requests.post(path(f'avatars/{user_id}/{name}')) 
r

<Response [200]>

In [5]:
villes = ['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
langues_associees = ['dutch','danish','spanish','french','italian','bulgarian','maltese','austrian','lithuanian']
autres_langues = ['belgian', 'croatian', 'cypriot', 'czech', 'estonian', 'finnish', 'german', 'greek', 'hungarian', 'irish', 'latvian', 'luxembourgish', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'swedish']
#langues_alphabetique=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']

In [6]:
requests_first_test = []

for j in range(14,0,-1):
    for i in range(len(villes)):
            params = {
                "avatar_name": name,
                "language": langues_associees[i],
                "city": villes[i],
                "date": j,
                "mobile": random.randint(0,1),
            }
            r1 = requests.get(path(f"pricing/{user_id}"), params=params)
            r1.json()
            requests_first_test.append(r1)

            params = {
                "avatar_name": name,
                "language": random.choice(autres_langues),
                "city": villes[i],
                "date": j,
                "mobile": random.randint(0,1),
            }
            r2 = requests.get(path(f"pricing/{user_id}"), params=params)
            r2.json()

            requests_first_test.append(r2)
                

#print(requests_first_test)

In [7]:
#build a training set

pricing_requests = []

for r in requests_first_test:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,918,336,23,amsterdam,14,dutch,0,59852
1,442,190,19,amsterdam,14,dutch,0,59852
2,588,173,0,amsterdam,14,dutch,0,59852
3,807,323,17,amsterdam,14,dutch,0,59852
4,271,81,0,amsterdam,14,dutch,0,59852


In [9]:
hotels = pd.read_csv('C:/Users/UserENM/Documents/INSA/ia/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,918,336,23,amsterdam,14,dutch,0,59852,Morriott International,Corlton,1,1,0
1,442,190,19,amsterdam,14,dutch,0,59852,Accar Hotels,Marcure,0,0,0
2,588,173,0,amsterdam,14,dutch,0,59852,Independant,Independant,0,0,0
3,807,323,17,amsterdam,14,dutch,0,59852,Chillton Worldwide,Chill Garden Inn,0,1,0
4,271,81,0,amsterdam,14,dutch,0,59852,Independant,Independant,1,0,0


In [10]:
pricing_requests.to_csv('pricing_requests_first_test.csv', index=False)


<div class="alert alert-success">
<b>Test 2</b>
</div>

On teste l'influence du nom de l'avatar : pour une même ville on choisit un avatar (ex : avatar_paris ) on fait varier le nombre de jour (on commence à J-44 jusqu'à J-1). On fixe le mobile (téléphone dans un premier temps) et la langue (une langue aléatoire pour chaque utilisateur).

In [19]:
villes = ['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
langues_alphabetique=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']

In [21]:
requests_test2 = []

for i in villes : 
    name = 'avatar_'+i
    r = requests.post(path(f'avatars/{user_id}/{name}')) 
    langue = random.choice(langues_alphabetique)

    for j in range(44,0,-1):
        params = {
                    "avatar_name": name,
                    "language": langue,
                    "city": i,
                    "date": j,
                    "mobile": 1,
                }
        r1 = requests.get(path(f"pricing/{user_id}"), params=params)
        r1.json()
        requests_test2.append(r1)

In [22]:
#build a training set

pricing_requests2 = []

for r in requests_test2:
    pricing_requests2.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests2 = pd.concat(pricing_requests2)
pricing_requests2.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,223,134,139,amsterdam,44,spanish,1,59854
1,163,155,195,amsterdam,44,spanish,1,59854
2,9,74,140,amsterdam,44,spanish,1,59854
3,603,217,277,amsterdam,44,spanish,1,59854
4,140,98,111,amsterdam,44,spanish,1,59854


In [23]:
hotels = pd.read_csv('C:/Users/UserENM/Documents/INSA/ia/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests2 = pricing_requests2.join(hotels, on=['hotel_id', 'city'])
pricing_requests2.head()


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,223,134,139,amsterdam,44,spanish,1,59854,Boss Western,J.Halliday Inn,0,0,0
1,163,155,195,amsterdam,44,spanish,1,59854,Yin Yang,Royal Lotus,0,1,0
2,9,74,140,amsterdam,44,spanish,1,59854,Independant,Independant,0,0,0
3,603,217,277,amsterdam,44,spanish,1,59854,Morriott International,Corlton,0,1,0
4,140,98,111,amsterdam,44,spanish,1,59854,Boss Western,Boss Western,0,0,0


In [24]:
pricing_requests2.to_csv('pricing_requests_test2.csv', index=False)


<div class="alert alert-success">
<b>Test 3</b>
</div>

On refait le même test mais avec un ordinateur

In [25]:
requests_test3 = []

for i in villes : 
    name = 'avatar_ordi'+i
    r = requests.post(path(f'avatars/{user_id}/{name}')) 
    langue = random.choice(langues_alphabetique)

    for j in range(44,0,-1):
        params = {
                    "avatar_name": name,
                    "language": langue,
                    "city": i,
                    "date": j,
                    "mobile": 0,
                }
        r1 = requests.get(path(f"pricing/{user_id}"), params=params)
        r1.json()
        requests_test3.append(r1)

In [26]:
#build a training set

pricing_requests3 = []

for r in requests_test3:
    pricing_requests3.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests3 = pd.concat(pricing_requests3)
pricing_requests3.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,529,230,281,amsterdam,44,austrian,0,59865
1,264,73,56,amsterdam,44,austrian,0,59865
2,809,225,277,amsterdam,44,austrian,0,59865
3,752,235,280,amsterdam,44,austrian,0,59865
4,660,136,140,amsterdam,44,austrian,0,59865


In [27]:
hotels = pd.read_csv('C:/Users/UserENM/Documents/INSA/ia/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests3 = pricing_requests3.join(hotels, on=['hotel_id', 'city'])
pricing_requests3.head()


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,529,230,281,amsterdam,44,austrian,0,59865,Morriott International,Corlton,1,0,1
1,264,73,56,amsterdam,44,austrian,0,59865,Independant,Independant,1,0,0
2,809,225,277,amsterdam,44,austrian,0,59865,Chillton Worldwide,Chill Garden Inn,0,1,0
3,752,235,280,amsterdam,44,austrian,0,59865,Accar Hotels,Safitel,0,0,0
4,660,136,140,amsterdam,44,austrian,0,59865,Chillton Worldwide,Tripletree,0,0,0


In [28]:
pricing_requests3.to_csv('pricing_requests_test3.csv', index=False)


<div class="alert alert-success">
<b>Test 4</b>
</div>

On cherche l'importance de la situation géographique de l'hotel. On coupe l'Europe en deux : l'Europe de l'est et de l'ouest. Si on avait voulu classer les hotels selon le PIB par habitant du pays (avec la "séparation" le PIB par habitant de l'Europe des 27), on aurait eu la même séparation (à l'exception de l'Espagne). Ainsi, Sofia et Vilnius sont les hotels de l'Europe de l'est et les autres ceux de l'Europe de l'ouest.

On prend une ville aléatoirement en Europe de l'ouest puis de l'est. On prend un utilisateur aléatoirement, un appareil aléatoire, on fixe le nombre de jours (on regarde j-7 puis j-14 puis j-20 puis j-40) et on fixe l'interface (on teste par exemple avec une interface française)


In [12]:
europe_ouest=['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'valletta', 'vienna']
europe_est=['sofia', 'vilnius']

In [14]:
requests_test4 = []
jours=[40,20,14,7]

for i in range(70):
    for j in jours:
            caracteres = "azertyuiopqsdfghjklmwxcvbnAZERTYUIOPQSDFGHJKLMWXCVBN0123456789" #Tu peux en ajouter si tu veux
            longueur = 10 #10 est un exemple tu mets le nombre que tu veux.
            nom = "" #Variable mot de passe
            compteur = 0 #Compteur de lettres
 
            while compteur < longueur:
                lettre = caracteres[random.randint(0, len(caracteres)-1)] #On tire au hasard une lettre
                nom += lettre #Ona joute la lettre au mot de passe
                compteur += 1 #On incrémente le compteur de lettres
            name = nom
            r = requests.post(path(f'avatars/{user_id}/{name}'))    
            params = {
                "avatar_name": nom,
                "language": 'french',
                "city": random.choice(europe_ouest),
                "date": j,
                "mobile": random.randint(0,1),
            }
            r1 = requests.get(path(f"pricing/{user_id}"), params=params)
            r1.json()
            requests_test4.append(r1)

            
for i in range(20):
    for j in jours:
            caracteres = "azertyuiopqsdfghjklmwxcvbnAZERTYUIOPQSDFGHJKLMWXCVBN0123456789" #Tu peux en ajouter si tu veux
            longueur = 10 #10 est un exemple tu mets le nombre que tu veux.
            nom = "" #Variable mot de passe
            compteur = 0 #Compteur de lettres
 
            while compteur < longueur:
                lettre = caracteres[random.randint(0, len(caracteres)-1)] #On tire au hasard une lettre
                nom += lettre #Ona joute la lettre au mot de passe
                compteur += 1 #On incrémente le compteur de lettres
            name = nom
            r = requests.post(path(f'avatars/{user_id}/{name}'))                
            params = {
                "avatar_name": nom,
                "language": 'french',
                "city": random.choice(europe_est),
                "date": j,
                "mobile": random.randint(0,1),
            }
            r2 = requests.get(path(f"pricing/{user_id}"), params=params)
            r2.json()
            requests_test4.append(r2)            


#print(requests_first_test)

In [16]:
#build a training set

pricing_requests4 = []

for r in requests_test4:
    pricing_requests4.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests4 = pd.concat(pricing_requests4)
pricing_requests4.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,160,92,43,valletta,40,french,1,70148
1,200,130,106,valletta,40,french,1,70148
2,334,76,45,valletta,40,french,1,70148
3,501,71,42,valletta,40,french,1,70148
4,985,65,91,valletta,40,french,1,70148


In [18]:
hotels = pd.read_csv('/home/enora/Documents/insa/IA/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests4 = pricing_requests4.join(hotels, on=['hotel_id', 'city'])
pricing_requests4.head()


,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,160,92,43,valletta,40,french,1,70148,Independant,Independant,1,0,0
1,200,130,106,valletta,40,french,1,70148,Boss Western,J.Halliday Inn,0,0,0
2,334,76,45,valletta,40,french,1,70148,Yin Yang,8 Premium,0,0,0
3,501,71,42,valletta,40,french,1,70148,Yin Yang,8 Premium,1,0,0
4,985,65,91,valletta,40,french,1,70148,Independant,Independant,1,0,0


In [19]:
pricing_requests4.to_csv('pricing_requests_test4.csv', index=False)


<div class="alert alert-success">
<b>Autre test</b>
</div>

test 6 : J-7 à J0,  hungarian, finnish, austrian, romanian, slovakian, swedish, estonian, bulgarian et danish,  Amsterdam, Paris et Madrid

In [4]:
#Création de l'avatar
name = 'av1'
r = requests.post(path(f'avatars/{user_id}/{name}')) 
r

<Response [200]>

In [5]:
villes = ['amsterdam', 'madrid', 'paris']
langues_associees = ['danish','bulgarian','austrian','estonian', 'finnish','hungarian','romanian', 'slovakian','swedish']

In [7]:
requests_6 = []

for k in range(5):
    for j in range(7,0,-1):
        for i in range(len(langues_associees)):
                params = {
                    "avatar_name": name,
                    "language": langues_associees[i],
                    "city": random.choice(villes),
                    "date": j,
                    "mobile": random.randint(0,1),
                }
                r1 = requests.get(path(f"pricing/{user_id}"), params=params)
                r1.json()
                requests_6.append(r1)

                

In [56]:
pricing_requests = []
for r in requests_test:
    try :
        pricing_requests.append(pd.DataFrame(r.json()['prices']).assign(**r.json()['request']))
    except :
        print(r)
pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422]>
<Response [422

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,495,366,7,madrid,7,danish,1,313004
1,312,117,0,madrid,7,danish,1,313004
2,418,159,0,madrid,7,danish,1,313004
3,185,156,0,madrid,7,danish,1,313004
4,439,115,0,madrid,7,danish,1,313004


Trop de requêtes ont été faites, donc tout n'a pas marché

In [57]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,495,366,7,madrid,7,danish,1,313004
1,312,117,0,madrid,7,danish,1,313004
2,418,159,0,madrid,7,danish,1,313004
3,185,156,0,madrid,7,danish,1,313004
4,439,115,0,madrid,7,danish,1,313004
...,...,...,...,...,...,...,...,...
116,507,121,1,amsterdam,1,estonian,1,313004
117,557,117,0,amsterdam,1,estonian,1,313004
118,157,386,1,amsterdam,1,estonian,1,313004
119,10,158,0,amsterdam,1,estonian,1,313004


In [59]:
hotels = pd.read_csv('/home/enora/Documents/insa/IA/defi_IA-main/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,495,366,7,madrid,7,danish,1,313004,Chillton Worldwide,Chill Garden Inn,0,0,0
1,312,117,0,madrid,7,danish,1,313004,Independant,Independant,1,1,0
2,418,159,0,madrid,7,danish,1,313004,Boss Western,J.Halliday Inn,1,0,0
3,185,156,0,madrid,7,danish,1,313004,Morriott International,CourtYord,0,0,0
4,439,115,0,madrid,7,danish,1,313004,Boss Western,Boss Western,1,0,0


In [60]:
pricing_requests.to_csv('pricing_requests_test7.csv', index=False)

<div class="alert alert-success">
<b>Autres test</b>
</div>

On fait un maximum de random au hasard pour fournir notre base de données pour obtenir une application gradio performante

<div class="alert alert-success">
<b>Soumission</b>
</div>

In [ ]:
# test sur le jeu de données test pour la soumission
file_test = '/home/enora/Documents/insa/IA/defi_IA-main/test_set.csv'
hotel_features = pd.read_csv('/home/enora/Documents/insa/IA/defi_IA-main/features_hotels.csv',index_col=['hotel_id', 'city'])

dataset_test = pd.DataFrame(pd.read_csv(file_test,index_col=0))
X_dataset_test= dataset_test.drop(labels=['order_requests'], axis=1)
X_dataset_test = X_dataset_test.join(hotel_features, on=['hotel_id', 'city'])


# Variables explicatives

#encodage avec OneHotEncoder

enc_data_test=pd.DataFrame(enc.fit_transform(X_dataset_test[["city","language","group","brand"]]).toarray())
df_test_ohe=dataset.join(enc_data_test)
one_hot_encoded_data_test = pd.get_dummies(X_dataset_test, columns = ["city","language","group","brand"])


In [ ]:
one_hot_encoded_data_test_reorder = one_hot_encoded_data_test[np.array(X_ohe.columns)] # on réarrange les colonnes de sorte à ce qu'elles soient dans le même ordre que pour l'entrainement
one_hot_encoded_data_test_reorder

In [ ]:
y_pred = xgb.predict(one_hot_encoded_data_test_reorder)
y_pred=pd.DataFrame(y_pred)
y_pred.to_csv("submission5.csv")